In [16]:
import pandas as pd
from mendeleev import get_engine

* elements
* ionicradii
* ionizationenergies
* oxidationstates
* groups
* series
* isotopes

## Retrieve all the data on the elements and put it pandas DataFrame

In [3]:
engine = get_engine()

In [12]:
ptable = pd.read_sql('elements', engine)

In [15]:
ptable.columns

Index([              u'annotation',            u'atomic_number',
                  u'atomic_radius',            u'atomic_volume',
                          u'block',            u'boiling_point',
                u'covalent_radius',                  u'density',
                    u'description',    u'dipole_polarizability',
              u'electron_affinity',        u'electronegativity',
       u'electronic_configuration',         u'evaporation_heat',
                    u'fusion_heat',                 u'group_id',
               u'lattice_constant',        u'lattice_structure',
                           u'mass',            u'melting_point',
                           u'name',                   u'period',
                      u'series_id',            u'specific_heat',
                         u'symbol',     u'thermal_conductivity',
                     u'vdw_radius'],
      dtype='object')

In [20]:
isotopes = pd.read_sql('isotopes', engine, index_col='id')

In [21]:
isotopes

,atomic_number,mass,abundance,mass_number
id,,,,
1,1,1.007825,0.999885,1
2,1,2.014102,0.000115,2
3,2,3.016029,0.000001,3
4,2,4.002603,0.999999,4
5,3,6.015122,0.075900,6
6,3,7.016004,0.924100,7
7,4,9.012182,1.000000,9
8,5,10.012937,0.199000,10
9,5,11.009305,0.801000,11


### Merge the elements table with the ionic radii table

In [9]:
table = pd.merge(ptable, ionicrad, how='outer', on='atomic_number')

Print all the ionic radii for silver

In [13]:
table[table['symbol'] == 'Ag'][cols]#[['symbol', 'charge', 'coordination', 'crystal_radius', 'ionic_radius']]

,id,atomic_number,charge,econf,coordination,spin,crystal_radius,ionic_radius,origin,most_reliable
228,2,47,1,4d10,II,,0.81,0.67,,False
229,3,47,1,4d10,IV,,1.14,1.00,"calculated,",False
230,4,47,1,4d10,IVSQ,,1.16,1.02,,False
231,5,47,1,4d10,V,,1.23,1.09,"calculated,",False
232,6,47,1,4d10,VI,,1.29,1.15,"calculated,",False
233,7,47,1,4d10,VII,,1.36,1.22,,False
234,8,47,1,4d10,VIII,,1.42,1.28,,False
235,9,47,2,4d9,IVSQ,,0.93,0.79,,False
236,10,47,2,4d9,VI,,1.08,0.94,,False
237,11,47,3,4d8,IVSQ,,0.81,0.67,,False


In [ ]:
ionicrad = pd.read_sql('ionicradii', engine)
cols = ionicrad.columns